In [1]:
import os
import re
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
path = os.path.dirname(os.path.abspath('Preprocess_Transients.ipynb'))
total_files=list()
cleaned_files=list()
ids=list()

In [3]:
for file in os.listdir("light_curves_fausnaugh/light_curves_fausnaugh"):
    total_files.append(file)
    if bool(re.search(r"\blc_\d+[a-zA-Z]{1,5}_cleaned\b", file)):
        cleaned_files.append(file)
        ids.extend(re.findall("_(.*?)_cleaned", file)) 

In [4]:
print(len(cleaned_files), len(total_files), len(ids))

3836 3857 3836


In [5]:
diff = set(total_files)-set(cleaned_files)

In [6]:
diff

{'lc_2018fhw_detrended_cleaned',
 'lc_2018fvi_detrended_cleaned',
 'lc_2018hib_detrended_cleaned',
 'lc_2018hss_detrended_cleaned',
 'lc_2018kfv_detrended_cleaned',
 'lc_2019mue_detrended-detrended_cleaned',
 'lc_2019mue_detrended_cleaned',
 'lc_2019pnw_detrended_cleaned',
 'lc_2019prw_detrended_cleaned',
 'lc_2019ptv_detrended_cleaned',
 'lc_2019qmm_detrended_cleaned',
 'lc_2019qsc_detrended_cleaned',
 'lc_2019ulr_detrended_cleaned',
 'lc_2020kbq_detrended_cleaned',
 'lc_2020npk_detrended_cleaned',
 'lc_2020to_detrended_cleaned',
 'lc_2020wd_detrended_cleaned',
 'lc_2020zjf_detrended_cleaned',
 'lc_2021bix_detrended_cleaned',
 'lc_2021caf_detrended_cleaned',
 'lc_GRB191016a_cleaned'}

In [15]:
data = pd.read_csv(path+f"/light_curves_fausnaugh/light_curves_fausnaugh/{cleaned_files[0]}", sep=r'\s+')

In [16]:
data.head(10)

,BTJD,TJD,cts,e_cts,bkg,bkg_model,bkg2,e_bkg2
0,2552.85815,2552.85424,-3462.3304,1272.4220,838.0194,-1265.4269,-1096.1326,1272.4220
1,2552.86510,2552.86119,1297.6085,1302.4403,4687.8755,-1264.7468,63.9336,1302.4403
2,2552.87204,2552.86813,-1494.9047,1318.4532,9010.4725,-1264.7468,-622.1638,1318.4532
3,2552.87898,2552.87507,10.0074,1486.7589,7895.4045,-1096.1326,1654.1474,1486.7589
4,2552.88593,2552.88202,-3648.6368,1532.6629,15694.5643,-1096.1326,-1589.0866,1532.6629
5,2552.89288,2552.88896,-1449.0730,1489.2339,11503.3610,-1096.1326,364.3844,1489.2339
6,2552.89982,2552.89591,1299.9789,1274.1106,4990.6856,-1096.1326,-226.7938,1274.1106
7,2552.90677,2552.90285,589.8731,1145.5637,9022.0823,-1096.1326,-1979.6934,1145.5637
8,2552.91371,2552.90980,-1364.8583,1130.0365,11651.5423,-1107.0297,-1264.7468,1130.0365
9,2552.92066,2552.91674,-1345.0779,1146.0881,8686.4337,-1096.1326,-2198.7152,1146.0881


In [17]:
adj_flux = data['cts']-data['bkg_model']

In [18]:
adj_flux

0      -2196.9035
1       2562.3553
2       -230.1579
3       1106.1400
4      -2552.5042
          ...    
3394    1232.3317
3395     793.2981
3396      74.5694
3397     116.0111
3398    -102.0144
Length: 3399, dtype: float64

In [19]:
cleaned_files[0]

'lc_2022hr_cleaned'

In [20]:
lcs = pd.DataFrame(columns=['phase', 'flux', 'flux_err'])

In [22]:
lcs['phase'] = data['BTJD']
lcs['flux'] = data['cts']-data['bkg_model']
lcs['flux_err'] = data['e_cts']

In [23]:
lcs.head()

,phase,flux,flux_err
0,2552.85815,-2196.9035,1272.4220
1,2552.86510,2562.3553,1302.4403
2,2552.87204,-230.1579,1318.4532
3,2552.87898,1106.1400,1486.7589
4,2552.88593,-2552.5042,1532.6629


In [ ]:
def preprocess_transient_data():
    
    path = os.path.dirname(os.path.abspath('Preprocess_Transients.ipynb'))
    total_files=list()
    cleaned_files=list()
    ids=list()
    
    if not os.path.exists('transients_data'):
            os.makedirs('transients_data')
            
            
    for file in os.listdir("light_curves_fausnaugh/light_curves_fausnaugh"):
        total_files.append(file)
        if bool(re.search(r"\blc_\d+[a-zA-Z]{1,5}_cleaned\b", file)):
            cleaned_files.append(file)
            ids.extend(re.findall("_(.*?)_cleaned", file)) 
            
    for i, file in enumerate(cleaned_files):
        data = pd.read_csv(path+f"/light_curves_fausnaugh/light_curves_fausnaugh/{file}", sep=r'\s+')
        lcs = pd.DataFrame(columns=['phase', 'flux', 'flux_err'])
        lcs['phase'] = data['BTJD']
        lcs['flux'] = data['cts']-data['bkg_model']
        lcs['flux_err'] = data['e_cts']
        
        transient_path = path + f'/transients_data/{ids[i]}.csv.gz'
        lcs.to_csv(path_or_buf=transient_path, header=True, index=False, compression='gzip')
        
    return
    
    